                            Import Libraries

In [26]:
from bs4 import BeautifulSoup as bs
import requests

In [24]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace('\xa0',' ') for li in row_data.find_all('li')]
    else:
        return row_data.get_text(" ", strip=True).replace('\xa0',' ')

In [57]:
#clean data to remove [1] [2], date format eg (09-25-1953)
def clean_tags(soup):
    for tag in soup.find_all(["sup","span"]):
        tag.decompose()

In [25]:
def get_info_box(url):

    r =  requests.get(url)
    soup = bs(r.content)


    info_box = soup.find(class_='infobox vevent')
    info_row = info_box.find_all("tr")

    clean_tags(soup)

    movie_info ={}
    for index,row in enumerate(info_row):
        if index==0:
            movie_info['Title'] = row.find("th").get_text(" ", strip=True)
        elif index==1:
            continue
        else:
            contenet_key = row.find("th").get_text(" ", strip=True)
            contenet_value = get_content_value(row.find("td"))
            movie_info[contenet_key] = contenet_value

    return movie_info

In [37]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup = bs(r.content)
contenets = soup.prettify()

In [49]:
movies = soup.select('.wikitable.sortable i')
movie_info_list =[]
base_url = 'https://en.wikipedia.org'

for index,movies in enumerate(movies):
    if index==2:
        break 
    try:
        relative_path = movies.a['href']
        full_path = base_url+relative_path
        title = movies.a['title']

        movie_info_list.append(get_info_box(full_path))
        
        # print(relative_path)
        # print(title)
        # print()
        
    except Exception as e:
        print(movies.get_text(), e)

Zorro the Avenger 'NoneType' object has no attribute 'find'
The Sign of Zorro 'NoneType' object has no attribute 'find'
Escape from the Dark 'NoneType' object is not subscriptable
Spirited Away 'NoneType' object has no attribute 'get_text'
Howl's Moving Castle 'NoneType' object has no attribute 'get_text'
The Nightmare Before Christmas 3D 'NoneType' object has no attribute 'get_text'
The Secret of the Magic Gourd 'NoneType' object has no attribute 'get_text'
Trail of the Panda 'NoneType' object is not subscriptable
Ponyo 'NoneType' object has no attribute 'get_text'
Tales from Earthsea 'NoneType' object has no attribute 'get_text'
Anaganaga O Dheerudu 'NoneType' object has no attribute 'get_text'
The Secret World of Arrietty 'NoneType' object has no attribute 'get_text'
Tini: The Movie 'NoneType' object has no attribute 'get_text'
Growing Up Wild 'NoneType' object is not subscriptable
Born in China 'NoneType' object has no attribute 'get_text'
Expedition China 'NoneType' object is not 

In [50]:
movie_info_list

[{'Title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['David Hand',
   'William Cottrell',
   'Wilfred Jackson',
   'Larry Morey',
   'Perce Pearce',
   'Ben Sharpsteen'],
  'Written by': ['Ted Sears',
   'Richard Creedon',
   'Otto Englander',
   'Dick Rickard',
   'Earl Hurd',
   'Merrill De Maris',
   'Dorothy Ann Blank',
   'Webb Smith'],
  'Based on': 'Snow White by The Brothers Grimm',
  'Produced by': 'Walt Disney',
  'Starring': ['Adriana Caselotti',
   'Lucille La Verne',
   'Harry Stockwell',
   'Roy Atwell',
   'Pinto Colvig',
   'Otis Harlan',
   'Scotty Mattraw',
   'Billy Gilbert',
   'Eddie Collins',
   'Moroni Olsen',
   'Stuart Buchanan'],
  'Music by': ['Frank Churchill', 'Paul Smith', 'Leigh Harline'],
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': ['December 21, 1937 ( 1937-12-21 ) ( Carthay Circle Theatre )',
   'February 4, 1938 ( 1938-02-04 ) (United States)'],
  'Running time': '83 minute

                                                            Save Data to JSON

In [51]:
import json

In [52]:
def save_data(title,data):
    with open(title,'w', encoding='utf-8') as f:
        json.dump(data,f,ensure_ascii=False, indent=2)

In [53]:
def load_data(title):
    with open(title,encoding='utf-8') as f:
        return json.load(f)

In [54]:
save_data('Disney Dataset',movie_info_list)

                                                            Clean Data